In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)


In [9]:
df_sup=pd.read_csv("../data/data_raw/superstore.csv")

In [10]:
df_sup.sample()

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,Order.Priority,Product.ID,Product.Name,Profit,Quantity,Region,Row.ID,Sales,Segment,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum
46122,Office Supplies,Houston,United States,CA-120554,Cathy Armstrong,0.8,US,1,2012-09-05 00:00:00.000,CA-2012-162621,Medium,OFF-BI-10003708,Acco Four Pocket Poly Ring Binder with Label H...,-7.8225,3,Central,35786,4,Home Office,2012-09-11 00:00:00.000,Standard Class,0.14,Texas,Binders,2012,North America,36


In [11]:
df_ind = pd.read_excel("../data/data_raw/indicators.xlsx")  

# Cambio nombre de algunas columnas para luego hacer el join con el otro dataset
df_ind.rename(columns={"Country Name": "Country", "Time": "Year",
                           "Inflation, consumer prices (annual %) [FP.CPI.TOTL.ZG]": "Inflation(%)",
                            "Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]": "Exports_GDP(%)",
                            "Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]": "Imports_GDP(%)",
                            "GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]": "GDP_Growth(%)"}, 
                            inplace=True)  

In [12]:
df_ind.sample(3)

,Year,Time Code,Country,Country Code,Inflation(%),Exports_GDP(%),Imports_GDP(%),GDP_Growth(%)
747,2014,YR2014,Italy,ITA,0.241047,28.622403,25.832127,-0.914683
600,2013,YR2013,Saudi Arabia,SAU,3.532525,51.42091,30.496301,-2.866422
815,2014,YR2014,San Marino,SMR,1.112963,..,..,0.471382


In [13]:
df_merged = pd.merge(df_sup, df_ind, on=['Country', 'Year'], how='left')  

In [14]:
# Ver los países únicos en cada dataset
sup_countries = set(df_sup['Country'].unique())
ind_countries = set(df_ind['Country'].unique())

missing_countries = sup_countries - ind_countries
print("Países en Superstore que no tienen coincidencia en el dataset Indicadores:\n", missing_countries)


Países en Superstore que no tienen coincidencia en el dataset Indicadores:
 {'Egypt', 'Iran', 'Democratic Republic of the Congo', 'Myanmar (Burma)', 'South Korea', 'Venezuela', 'Czech Republic', 'Swaziland', 'Republic of the Congo', 'Kyrgyzstan', 'Martinique', 'Yemen', 'Turkey', 'Taiwan', 'Russia', 'Hong Kong', 'Macedonia', 'Slovakia', 'Guadeloupe', 'Vietnam', 'Syria'}


In [15]:
country_corrections = {
    'Macedonia': 'North Macedonia',
    'Venezuela': 'Venezuela, RB',
    'South Korea': 'Korea, Rep.',
    'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
    'Republic of the Congo': 'Congo, Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    'Swaziland' : 'Eswatini', 
    'Russia': 'Russian Federation',
    'Hong Kong': 'Hong Kong SAR, China', 
    'Egypt': 'Egypt, Arab Rep.',
    'Czech Republic': 'Czechia',
    'Iran': 'Iran, Islamic Rep.',
    'Turkey': 'Turkiye',
    'Yemen': 'Yemen, Rep.',
    'Myanmar (Burma)': 'Myanmar',
    'Syria': 'Syrian Arab Republic',
    'Slovakia': 'Slovak Republic',
    'Vietnam': 'Viet Nam'   
}

df_sup['Country'] = df_sup['Country'].replace(country_corrections)


In [16]:
df_merged = pd.merge(df_sup, df_ind, on=['Country', 'Year'], how='left')  

In [17]:
# Creo un dataframe alternativo para trabajar
df = df_merged.copy()

In [18]:
# Elimino países que no tienen match con el dataset Indicadores
paises_a_eliminar = ["Guadeloupe", "Taiwan", "Martinique"]
df = df[~df["Country"].isin(paises_a_eliminar)]

In [19]:
df['Year'] = df['Year'].astype(int)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51243 entries, 0 to 51289
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51243 non-null  object 
 1   City            51243 non-null  object 
 2   Country         51243 non-null  object 
 3   Customer.ID     51243 non-null  object 
 4   Customer.Name   51243 non-null  object 
 5   Discount        51243 non-null  float64
 6   Market          51243 non-null  object 
 7   记录数             51243 non-null  int64  
 8   Order.Date      51243 non-null  object 
 9   Order.ID        51243 non-null  object 
 10  Order.Priority  51243 non-null  object 
 11  Product.ID      51243 non-null  object 
 12  Product.Name    51243 non-null  object 
 13  Profit          51243 non-null  float64
 14  Quantity        51243 non-null  int64  
 15  Region          51243 non-null  object 
 16  Row.ID          51243 non-null  int64  
 17  Sales           51243 non-null  int6

In [21]:
df.sample(3)

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,Order.Priority,Product.ID,Product.Name,Profit,Quantity,Region,Row.ID,Sales,Segment,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum,Time Code,Country Code,Inflation(%),Exports_GDP(%),Imports_GDP(%),GDP_Growth(%)
49598,Technology,Plantation,United States,AH-106904,Anna Häberlin,0.2,US,1,2012-11-12 00:00:00.000,CA-2012-113628,Medium,TEC-PH-10000441,VTech DS6151,10.0792,1,South,35107,101,Corporate,2012-11-19 00:00:00.000,Standard Class,5.96,Florida,Phones,2012,North America,46,YR2012,USA,2.069337,13.644051,17.037788,1.541466
38964,Technology,Panama City,Panama,TH-212353,Tiffany House,0.4,LATAM,1,2011-10-07 00:00:00.000,US-2011-112137,Medium,TEC-PH-10002867,"Cisco Smart Phone, with Caller ID",-510.3120,3,Central,6266,785,Corporate,2011-10-11 00:00:00.000,Standard Class,48.01,Panama,Phones,2011,LATAM,41,YR2011,PAN,5.875802,77.745672,80.462723,9.940362
5989,Office Supplies,Sandakan,Malaysia,CK-127601,Cyma Kinney,0.0,APAC,1,2011-08-18 00:00:00.000,IN-2011-33211,High,OFF-FA-10004698,"Stockwell Staples, Metal",0.6000,2,Southeast Asia,27035,21,Corporate,2011-08-22 00:00:00.000,Second Class,3.03,Sabah,Fasteners,2011,APAC,34,YR2011,MYS,3.174471,85.255442,69.682242,3.465974


In [23]:
# Guardo el archivo
df.to_csv('../data/data_processed/transformacion.csv', index=False)